# 1.   Исследование надежности заемщиков.

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
<br><br>
Имеющиеся признаки:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита
</div>

### 1.0.2.   Шаг 1. Обзор данных

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('/datasets/data.csv')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Рассмотрим данные чуть подробнее</div>

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
days_employed и total_income имеют пропуски, причем одинаковое количество</div>

In [6]:
days_employed_null = data['days_employed'].isnull().sum()
days_employed_total = len(data['days_employed'])
print(f'Пропусков в days_employed {round(days_employed_null / days_employed_total * 100, 2)}%')

Пропусков в days_employed 10.1%


In [7]:
total_income_null = data['total_income'].isnull().sum()
total_income_total = len(data['total_income'])
print(f'Пропусков в total_income {round(total_income_null / total_income_total * 100, 2)}%')

Пропусков в total_income 10.1%


In [8]:
display(data[data.days_employed.isnull()].head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [9]:
display(data[data.total_income.isnull()].tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Судя по всему, некоторые не указывают стаж работы, отсюда пропуски в days_employed</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Есть вероятность, что пропуски в одних и тех же строках: логично, что у неработающего человека будет отсутствовать доход</div>

In [10]:
# сравним индексы строк с пропущенными значениями в обоих столбцах
list(data[data.days_employed.isnull()].index) == list(data[data.total_income.isnull()].index)

True

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Действительно, пропуски встречаются только вместе</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Было бы неплохо удалить их, так как с данными о доходе, не зная стажа работы, работать трудно – его может не быть вовсе, а может быть очень большой, в обход налогов</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Но пропусков 10%, мы себе такую роскошь позволить не можем, следовательно, заполняем медианными значениями (также, можно еще ориентироваться на сферу деятельности)</div>

### 1.0.3.   Шаг 2.1 Заполнение пропусков

In [12]:
# Создаем функцию заполнения пропусков по сферам деятельности
def income_sphere(row):
    if pd.isna(row['total_income']):
        return data[data.income_type == row.income_type]['total_income'].median()
    else:
        return row['total_income']

In [13]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data.apply(income_sphere, axis=1)
display(data.days_employed.isna().sum())
display(data.total_income.isna().sum())

0

0

In [14]:
display(data.head())
display(data.tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Вышли отрицательные значения, с ними и будем разбираться</div>

### 1.0.4.   Шаг 2.2 Проверка данных на аномалии и исправления.

#### 1.0.4.1.   Общая проверка

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Посмотрим на основные данные, категориальные, где можно оценить ситуацию на глаз</div>

In [15]:
data_categorical_list = ['children', 'education', 'education_id', 'family_status', 
                         'family_status_id', 'gender', 'income_type', 'debt', 'purpose']

for categoria in data_categorical_list:
    print(categoria)
    print(data[categoria].unique())
    print()

children
[ 1  0  3  2 -1  4 20  5]

education
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

education_id
[0 1 2 3 4]

family_status
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

family_status_id
[0 1 2 3 4]

gender
['F' 'M' 'XNA']

income_type
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

debt
[0 1]

purpose
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего а

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Очень хочется посмотреть на семью счастливых XNA с -1 ребенком, но имеем что имеем</div>

#### 1.0.4.2.   Дети и гендер

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Оценим масштаб ошибок с детьми и полом</div>

In [16]:
anomal_rows = {'children': [-1, 20],
               'gender': ['XNA']}

for row in anomal_rows:
    print(row)
    print()
    for value in anomal_rows[row]:
        value_count = len(data[data[row] == value])
        print(f'{value}: {value_count} ({round(value_count / len(data[row]) * 100, 2)}%)')
    print()
    print()

children

-1: 47 (0.22%)
20: 76 (0.35%)


gender

XNA: 1 (0.0%)




<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Можно предположить, что -1 ребенок появился из 1 ребенка, а 20 – из 2</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Но так как их сравнительно незначительное количество, проще -1 заменить на 1, а 20 – заменить медианой, округленной вниз</div>

In [17]:
# Заменяем -1 детей на 1 детей
data['children'] = data['children'].replace(-1, 1)
print('-1 детей:', len(data[data['children'] == -1]))

# Заменяем 20 детей на mean детей
data['children'] = data['children'].replace(20, int(data['children'].mean()))
print('20 детей:', len(data[data['children'] == 20]))

-1 детей: 0
20 детей: 0


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
С неизвестным гендером работаем культурно и толерантно – убираем из выборки</div>

In [18]:
# Запомним сколько было
gender_count_old = len(data['gender'])
data = data[data['gender'] != 'XNA']
# Метод костыльный, так что перестраховываемся
gender_count_new = len(data['gender'])
print('Удалили гендеров:', gender_count_old-gender_count_new)

Удалили гендеров: 1


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Сверяемся:</div>

In [19]:
data_categorical_list = ['children', 'gender']

for categoria in data_categorical_list:
    print(categoria)
    print(data[categoria].unique())
    print()

children
[1 0 3 2 4 5]

gender
['F' 'M']



#### 1.0.4.3.   Образование

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Разберемся с образованием: там все проще, у нас явные дубликаты из-за регистра – приведем все к нижнему</div>

In [20]:
data['education_lower'] = data['education'].str.lower()
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,высшее
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,среднее
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,среднее
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,среднее
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,высшее
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,высшее
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,среднее
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,высшее
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,среднее


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Проверим новый столбец</div>

In [21]:
display(data['education_lower'].unique())

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### 1.0.5.   Шаг 2.7. Категоризация целей кредита.

#### 1.0.5.1.   Цель кредита

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Теперь разберемся с целью кредита</div>

In [22]:
for row in data['purpose'].unique():
    print(row)

покупка жилья
приобретение автомобиля
дополнительное образование
сыграть свадьбу
операции с жильем
образование
на проведение свадьбы
покупка жилья для семьи
покупка недвижимости
покупка коммерческой недвижимости
покупка жилой недвижимости
строительство собственной недвижимости
недвижимость
строительство недвижимости
на покупку подержанного автомобиля
на покупку своего автомобиля
операции с коммерческой недвижимостью
строительство жилой недвижимости
жилье
операции со своей недвижимостью
автомобили
заняться образованием
сделка с подержанным автомобилем
получение образования
автомобиль
свадьба
получение дополнительного образования
покупка своего жилья
операции с недвижимостью
получение высшего образования
свой автомобиль
сделка с автомобилем
профильное образование
высшее образование
покупка жилья для сдачи
на покупку автомобиля
ремонт жилью
заняться высшим образованием


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Попробуем хоть как то автоматизировать процесс</div>

In [23]:
# Создаем функциб с подстановкой значений, которые выделили на глаз (куски основных слов)

def purpose_dict_pred(row): 
    purposes = {
        'жил': 'недвижимость',
        'авто': 'автомобили',
        'образов': 'образование',
        'недвиж': 'недвижимость',
        'свад': 'свадьба'
    }
    for word in purposes:
        if word in row['purpose']:
            return purposes[word]

#### 1.0.5.2.   Добавление столбцов

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Дополняем и объединяем категории </div>

In [24]:
data['purpose_total'] = data.apply(purpose_dict_pred, axis=1)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,purpose_total
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,высшее,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,среднее,автомобили
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,среднее,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,среднее,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее,свадьба
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,высшее,недвижимость
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,высшее,недвижимость
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,среднее,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,высшее,свадьба
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,среднее,недвижимость


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Ну и раз уж мы здесь, приведем к божескому виду еще и гендеры со статусами</div>

In [25]:
data['gender_lower'] = data['gender'].str.lower()
data['family_status_lower'] = data['family_status'].str.lower()
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,purpose_total,gender_lower,family_status_lower
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,высшее,недвижимость,f,женат / замужем
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,среднее,автомобили,f,женат / замужем
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,среднее,недвижимость,m,женат / замужем
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,среднее,образование,m,женат / замужем
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее,свадьба,f,гражданский брак


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Проведем повторную проверку данных</div>

In [26]:
data_categorical_list = ['children', 'education_lower', 'family_status_lower', 
                         'gender_lower', 'purpose_total']

for categoria in data_categorical_list:
    print(categoria)
    print(data[categoria].unique())
    print()

children
[1 0 3 2 4 5]

education_lower
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']

family_status_lower
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']

gender_lower
['f' 'm']

purpose_total
['недвижимость' 'автомобили' 'образование' 'свадьба']



#### 1.0.5.3.   Числовые значения – общая проверка

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Переходим к числовым значениям, проверим их на адекватность</div>

In [27]:
digit_values = ['days_employed', 'dob_years', 'total_income', 'dob_years']
for column in digit_values:
    print(column)
    print('Минимум:', data[column].min())
    print('Максимум', data[column].max())
    print('Среднее', data[column].mean())
    print('Медиана', data[column].median())
    print('Ошибок:', data[column].isna().sum())
    print()

data[['days_employed', 'dob_years', 'total_income']].info()

days_employed
Минимум: -18388.949900568383
Максимум 401755.40047533
Среднее 56560.0729182367
Медиана -1203.369528770489
Ошибок: 0

dob_years
Минимум: 0
Максимум 75
Среднее 43.29427615684817
Медиана 42.0
Ошибок: 0

total_income
Минимум: 20667.26379327158
Максимум 2265604.028722744
Среднее 165223.5274577049
Медиана 142594.39684740017
Ошибок: 0

dob_years
Минимум: 0
Максимум 75
Среднее 43.29427615684817
Медиана 42.0
Ошибок: 0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 3 columns):
days_employed    21524 non-null float64
dob_years        21524 non-null int64
total_income     21524 non-null float64
dtypes: float64(2), int64(1)
memory usage: 672.6 KB


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Возраст клиентов выглядит не особо прилично, надо с ним разобраться</div>

#### 1.0.5.4.   Возраст клиентов

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Выведем все возрасты по возрастанию (неожиданная тавтология)</div>

In [28]:
print(sorted(list(data['dob_years'].unique())))

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Узнаем, сколько у нас 0-летних заемщиков</div>

In [29]:
print(len(data[data.dob_years == 0]))

101


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Немало. Но тут ничего нельзя поделать, узнать, сколько им реально лет не получится, то ли 20, то ли 50... Игнорируем!</div>

#### 1.0.5.5.   Изменение days_employed

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Хотел бы я работать отрицательное количество дней, но раз у меня нет такой возможности, лишим ее всех</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Отрицательные значения заменим на положительные, скорее всего ошибка выгрузки, хотя и не факт, но обратиться к грузчикам возможности нет, работаем с тем что есть</div>

In [30]:
# как обычно – спасает apply, int спокойно вернет отрицательное
data['days_employed_integer'] = data['days_employed'].apply(abs)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,purpose_total,gender_lower,family_status_lower,days_employed_integer
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,высшее,недвижимость,f,женат / замужем,8437.673028
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,среднее,автомобили,f,женат / замужем,4024.803754
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,среднее,недвижимость,m,женат / замужем,5623.422610
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,среднее,образование,m,женат / замужем,4124.747207
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее,свадьба,f,гражданский брак,340266.072047


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Похоже, тут все неплохо, посмотрим снова</div>

In [31]:
column = 'days_employed_integer'
print(column)
print('Минимум:', data[column].min())
print('Максимум', data[column].max())
print('Среднее', data[column].mean())
print('Медиана', data[column].median())

days_employed_integer
Минимум: 24.14163324048118
Максимум 401755.40047533
Среднее 60280.6488492364
Медиана 1807.8486042866098


In [32]:
print("401755 дней в годах это ~", int(401755 / 365), 'лет')

401755 дней в годах это ~ 1100 лет


In [33]:
display(data.sort_values(by='days_employed_integer').tail(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,purpose_total,gender_lower,family_status_lower,days_employed_integer
8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование,среднее,образование,f,женат / замужем,401590.452231
10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования,среднее,образование,f,в разводе,401591.828457
17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи,среднее,недвижимость,f,женат / замужем,401614.475622
13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу,среднее,свадьба,f,гражданский брак,401619.633298
4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости,среднее,недвижимость,f,женат / замужем,401635.032697
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба,среднее,свадьба,f,гражданский брак,401663.850046
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили,среднее,автомобили,m,женат / замужем,401674.466633
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем,среднее,недвижимость,f,женат / замужем,401675.093434
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования,высшее,образование,f,не женат / не замужем,401715.811749
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью,среднее,недвижимость,f,вдовец / вдова,401755.400475


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
И такая ситуация не одна, людей буквально держат в вечном рабстве и пора это исправить</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Есть подозрение, что 1100 лет это на самом деле 11 лет, но мы попробуем исхитриться и оставить только тех, кто работает дольше своей жизни после 18 лет</div>

In [34]:
data_zombie = data[data['days_employed_integer'] / 365 > data['dob_years']]
print(len(data_zombie['dob_years']))
display(data_zombie.head(10))

3529


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,purpose_total,gender_lower,family_status_lower,days_employed_integer
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее,свадьба,f,гражданский брак,340266.072047
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля,среднее,автомобили,f,вдовец / вдова,400281.136913
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью,среднее,недвижимость,f,не женат / не замужем,338551.952911
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости,среднее,недвижимость,m,женат / замужем,363548.489348
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью,среднее,недвижимость,f,женат / замужем,335581.668515
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы,среднее,свадьба,m,гражданский брак,394021.072184
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили,среднее,автомобили,f,женат / замужем,353731.432338
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование,среднее,образование,f,вдовец / вдова,370145.087237
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили,среднее,автомобили,f,женат / замужем,338113.529892
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем,высшее,автомобили,m,женат / замужем,359722.945074


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Пять с половиной тысяч несчастных, надо узнать, на сколько они переработали</div>

In [35]:
def pererabotka(row):
    return int(row['days_employed_integer'] / 365) - row['dob_years']

hight_work = data_zombie.apply(pererabotka, axis=1)
display(hight_work.head())

4      879
18    1043
24     870
25     929
30     857
dtype: int64

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Немало, узначем чуть глубже</div>

In [36]:
print('Минимум:', hight_work.min())
print('Максимум', hight_work.max())
print('Среднее', hight_work.mean())
print('Медиана', hight_work.median())

Минимум: 0
Максимум 1098
Среднее 918.1320487390195
Медиана 937.0


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
В целом, можно стандартно заменить средними значениями</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Возвращаем справедливость</div>

In [37]:
data_mean = data[data['days_employed_integer'] / 365 < data['dob_years']].mean()
data.loc[(data['days_employed_integer'] / 365 > data['dob_years']), 'days_employed_integer'] = data_mean

column = 'days_employed_integer'
print(column)
print('Минимум:', data[column].min())
print('Максимум', data[column].max())
print('Среднее', data[column].mean())
print('Медиана', data[column].median())

days_employed_integer
Минимум: 24.14163324048118
Максимум 18388.949900568383
Среднее 2215.391868119628
Медиана 1347.6449313585415


In [38]:
print("18388 дней в годах это ~", int(18388 / 365), 'лет')

18388 дней в годах это ~ 50 лет


In [39]:
display(data[data.days_employed_integer == 18388.949900568383])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,purpose_total,gender_lower,family_status_lower,days_employed_integer
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью,среднее,недвижимость,f,женат / замужем,18388.949901


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Странно начинать работать в 11 лет, но это уже чуть больше похоже на правду</div>

In [40]:
digit_values = ['days_employed_integer', 'dob_years', 'total_income']
for column in digit_values:
    print(column)
    print('Минимум:', data[column].min())
    print('Максимум', data[column].max())
    print('Среднее', data[column].mean())
    print('Медиана', data[column].median())
    print()

data[['days_employed', 'dob_years', 'total_income']].info()

days_employed_integer
Минимум: 24.14163324048118
Максимум 18388.949900568383
Среднее 2215.391868119628
Медиана 1347.6449313585415

dob_years
Минимум: 0
Максимум 75
Среднее 43.29427615684817
Медиана 42.0

total_income
Минимум: 20667.26379327158
Максимум 2265604.028722744
Среднее 165223.5274577049
Медиана 142594.39684740017

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 3 columns):
days_employed    21524 non-null float64
dob_years        21524 non-null int64
total_income     21524 non-null float64
dtypes: float64(2), int64(1)
memory usage: 672.6 KB


### 1.0.6.   Шаг 2.3. Изменение типов данных.

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Посмотрим какие у нас вообще есть типы данных</div>

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 17 columns):
children                 21524 non-null int64
days_employed            21524 non-null float64
dob_years                21524 non-null int64
education                21524 non-null object
education_id             21524 non-null int64
family_status            21524 non-null object
family_status_id         21524 non-null int64
gender                   21524 non-null object
income_type              21524 non-null object
debt                     21524 non-null int64
total_income             21524 non-null float64
purpose                  21524 non-null object
education_lower          21524 non-null object
purpose_total            21524 non-null object
gender_lower             21524 non-null object
family_status_lower      21524 non-null object
days_employed_integer    17995 non-null float64
dtypes: float64(3), int64(5), object(9)
memory usage: 3.0+ MB


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Кстати, многовато столбцов вышло, оставим только необходимые</div>

In [42]:
data_cool = data.drop(['days_employed', 'education', 'family_status', 'gender'], axis=1)

In [43]:
data_cool.head(10)

,children,dob_years,education_id,family_status_id,income_type,debt,total_income,purpose,education_lower,purpose_total,gender_lower,family_status_lower,days_employed_integer
0,1,42,0,0,сотрудник,0,253875.639453,покупка жилья,высшее,недвижимость,f,женат / замужем,8437.673028
1,1,36,1,0,сотрудник,0,112080.014102,приобретение автомобиля,среднее,автомобили,f,женат / замужем,4024.803754
2,0,33,1,0,сотрудник,0,145885.952297,покупка жилья,среднее,недвижимость,m,женат / замужем,5623.422610
3,3,32,1,0,сотрудник,0,267628.550329,дополнительное образование,среднее,образование,m,женат / замужем,4124.747207
4,0,53,1,1,пенсионер,0,158616.077870,сыграть свадьбу,среднее,свадьба,f,гражданский брак,NaN
5,0,27,0,1,компаньон,0,255763.565419,покупка жилья,высшее,недвижимость,m,гражданский брак,926.185831
6,0,43,0,0,компаньон,0,240525.971920,операции с жильем,высшее,недвижимость,f,женат / замужем,2879.202052
7,0,50,1,0,сотрудник,0,135823.934197,образование,среднее,образование,m,женат / замужем,152.779569
8,2,35,0,1,сотрудник,0,95856.832424,на проведение свадьбы,высшее,свадьба,f,гражданский брак,6929.865299
9,0,41,1,0,сотрудник,0,144425.938277,покупка жилья для семьи,среднее,недвижимость,m,женат / замужем,2188.756445


In [44]:
data_cool.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 13 columns):
children                 21524 non-null int64
dob_years                21524 non-null int64
education_id             21524 non-null int64
family_status_id         21524 non-null int64
income_type              21524 non-null object
debt                     21524 non-null int64
total_income             21524 non-null float64
purpose                  21524 non-null object
education_lower          21524 non-null object
purpose_total            21524 non-null object
gender_lower             21524 non-null object
family_status_lower      21524 non-null object
days_employed_integer    17995 non-null float64
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Пора привести total_income, dob_years и days_employed_integer к божескому виду</div>

In [45]:
data_cool['total_income'] = data_cool['total_income'].astype('int64')
display(data_cool['total_income'])

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21524, dtype: int64

In [46]:
data_cool['dob_years'] = data_cool['dob_years'].astype('int64')
display(data_cool['dob_years'])

0        42
1        36
2        33
3        32
4        53
         ..
21520    43
21521    67
21522    38
21523    38
21524    40
Name: dob_years, Length: 21524, dtype: int64

In [48]:
data_cool['days_employed_integer'].isna().sum()

3529

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Заполняем средними значениями</div>

In [49]:
fixed_days = data_cool['days_employed_integer'].fillna(data_cool['days_employed_integer'].mean())
data_cool['days_employed_integer'] = fixed_days

In [50]:
data_cool['days_employed_integer'].isna().sum()

0

In [51]:
data_cool['days_employed_integer'] = data_cool['days_employed_integer'].astype('int64')
display(data_cool['days_employed_integer'])

0        8437
1        4024
2        5623
3        4124
4        2215
         ... 
21520    4529
21521    2215
21522    2113
21523    3112
21524    1984
Name: days_employed_integer, Length: 21524, dtype: int64

In [52]:
data_cool.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 13 columns):
children                 21524 non-null int64
dob_years                21524 non-null int64
education_id             21524 non-null int64
family_status_id         21524 non-null int64
income_type              21524 non-null object
debt                     21524 non-null int64
total_income             21524 non-null int64
purpose                  21524 non-null object
education_lower          21524 non-null object
purpose_total            21524 non-null object
gender_lower             21524 non-null object
family_status_lower      21524 non-null object
days_employed_integer    21524 non-null int64
dtypes: int64(7), object(6)
memory usage: 2.3+ MB


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Выглядит культурно и прилично, а значит нам нравится. Оставляем</div>

### 1.0.8.   Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Разберемся с делами насущными – с образованием</div>

In [54]:
data_education_only = data_cool[['education_lower', 'education_id']].drop_duplicates().reset_index(drop=True)
display(data_education_only.head())

,education_lower,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
И с делами грядущими – семьей</div>

In [55]:
data_family_only = data_cool[['family_status_lower', 'family_status_id']].drop_duplicates().reset_index(drop=True)
display(data_family_only.head())

,family_status_lower,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Избавляемся от громоздких и ошибкоопасных столбцов в исходном датафрейме</div>

In [56]:
data_cool = data_cool.drop(['education_lower', 'family_status_lower'], axis=1)

In [57]:
data_cool.head()

,children,dob_years,education_id,family_status_id,income_type,debt,total_income,purpose,purpose_total,gender_lower,days_employed_integer
0,1,42,0,0,сотрудник,0,253875,покупка жилья,недвижимость,f,8437
1,1,36,1,0,сотрудник,0,112080,приобретение автомобиля,автомобили,f,4024
2,0,33,1,0,сотрудник,0,145885,покупка жилья,недвижимость,m,5623
3,3,32,1,0,сотрудник,0,267628,дополнительное образование,образование,m,4124
4,0,53,1,1,пенсионер,0,158616,сыграть свадьбу,свадьба,f,2215


### 1.0.9.   Шаг 2.6. Категоризация дохода.

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Создаем функцию, делаем apply</div>

In [58]:
def total_income_categorization(row):
    if row <= 30000:
        return 'E'
    if row <= 50000:
        return 'D'
    if row <= 200000:
        return 'C'
    if row <= 1000000:
        return 'B'
    return 'A'

data_cool['total_income_category'] = data_cool['total_income'].apply(total_income_categorization)
display(data_cool.head())

,children,dob_years,education_id,family_status_id,income_type,debt,total_income,purpose,purpose_total,gender_lower,days_employed_integer,total_income_category
0,1,42,0,0,сотрудник,0,253875,покупка жилья,недвижимость,f,8437,B
1,1,36,1,0,сотрудник,0,112080,приобретение автомобиля,автомобили,f,4024,C
2,0,33,1,0,сотрудник,0,145885,покупка жилья,недвижимость,m,5623,C
3,3,32,1,0,сотрудник,0,267628,дополнительное образование,образование,m,4124,B
4,0,53,1,1,пенсионер,0,158616,сыграть свадьбу,свадьба,f,2215,C


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
И вот они уже как на ладони</div>

### 1.0.10.   Пора исследовать

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Посмотрим на неплательщиков</div>

In [59]:
data_cool[data_cool.debt == 1].head()

,children,dob_years,education_id,family_status_id,income_type,debt,total_income,purpose,purpose_total,gender_lower,days_employed_integer,total_income_category
14,0,56,0,1,компаньон,1,165127,покупка жилой недвижимости,недвижимость,f,1844,C
32,0,34,1,1,сотрудник,1,139057,на проведение свадьбы,свадьба,f,4649,C
38,0,25,0,4,сотрудник,1,192247,образование,образование,m,597,C
55,0,54,1,1,пенсионер,1,118514,сыграть свадьбу,свадьба,f,1203,C
75,1,38,1,0,сотрудник,1,81935,операции с недвижимостью,недвижимость,m,2953,C


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Посмотрим на каждый параметр</div>

In [60]:
data_cool.pivot_table(index=['children'],
                      values=['debt'],
                      aggfunc='mean'
                     ).sort_values(by='debt')

,debt
children,
5,0.000000
0,0.075295
3,0.081818
1,0.091470
2,0.094404
4,0.097561


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Как видно, чаще всего не платят люди с 4 детьми, недалеко от них с 2 и с 1, после чего уже с отрывом идут с 3 и без детей. Абсолютно честные люди – с 5 детьми, скорее всего их просто было не так много</div>

In [61]:
data_cool.groupby('children')['debt'].agg(['count','sum','mean'])

,count,sum,mean
children,,,
0,14224,1071,0.075295
1,4865,445,0.091470
2,2055,194,0.094404
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [62]:
def my_mean(x): return x.mean()*100

# В кач-ве агрегирования можно любую функцию брать.
data_cool.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14224,1071,7.529528
1,4865,445,9.146968
2,2055,194,9.440389
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


In [63]:
def my_mean(x): return str(round(x.mean()*100,2)) +'%'

data_cool.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14224,1071,7.53%
1,4865,445,9.15%
2,2055,194,9.44%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.0%


In [64]:
def my_mean(x): return '{:.2%} '.format(x.mean())

data_cool.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14224,1071,7.53%
1,4865,445,9.15%
2,2055,194,9.44%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


In [65]:
data_cool.pivot_table(index=['family_status_id'],
                      values=['debt'],
                      aggfunc='mean'
                     ).sort_values(by='debt').merge(data_family_only, on='family_status_id')

,family_status_id,debt,family_status_lower
0,2,0.065625,вдовец / вдова
1,3,0.071130,в разводе
2,0,0.075202,женат / замужем
3,1,0.092912,гражданский брак
4,4,0.097405,не женат / не замужем


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Самые частые задержки у холостых, самые редкие – у товарищей с браком в прошлом</div>

In [66]:
data_cool.pivot_table(index=['total_income_category'],
                      values=['debt'],
                      aggfunc='mean'
                     ).sort_values(by='debt')

,debt
total_income_category,
D,0.060000
B,0.070621
A,0.080000
C,0.084546
E,0.090909


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Можно сделать вывод, что ниже всего задержки у категории D, хотя самые высокие показатели невозврата – у E</div>

In [67]:
data_cool.pivot_table(index=['purpose_total'],
                      values=['debt'],
                      aggfunc='mean'
                     ).sort_values(by='debt')

,debt
purpose_total,
недвижимость,0.072147
свадьба,0.079216
образование,0.091994
автомобили,0.093395


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Здесь можно составить рейтинг, который говорит о том, что самые честные – те кто работают с недвижимостью, а вот автомобили оставляют желать лучшего</div>

In [68]:
data_cool.pivot_table(index=['purpose'],
                      values=['debt'],
                      aggfunc='mean'
                     ).sort_values(by='debt')

,debt
purpose,
покупка своего жилья,0.054839
ремонт жилью,0.057190
строительство собственной недвижимости,0.066142
недвижимость,0.066246
покупка жилой недвижимости,0.067545
покупка недвижимости,0.069021
покупка жилья для семьи,0.070203
покупка коммерческой недвижимости,0.070783
жилье,0.071097


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Здесь видно, что получение дополнительного образования является самым частым кредитом с задержками выплат, и получение высшего тоже недалеко. Однако, сделки с недвижимостью все таки определились верно – у них самые слабые показатели невозврата</div>

### 1.0.11.   Ответы на вопросы.

###   Вопрос 1:

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Есть ли зависимость между количеством детей и возвратом кредита в срок?</div>

### Вывод 1:

<div style="background-color: #fff0ee; padding: 10px; font-family: monospace; font-size: 15px">
Да, люди с 4 детьми, 1 ребенком и вообще без детей реже возвращают кредиты</div>

### Вопрос 2:

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Есть ли зависимость между семейным положением и возвратом кредита в срок?</div>

### Вывод 2:

<div style="background-color: #fff0ee; padding: 10px; font-family: monospace; font-size: 15px">
Да, самые частые задержки – у людей, не закрепивших свой брак, а самые редкие – у тех, кто был со своей половинкой в прошлом</div>

### Вопрос 3:

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</div>

### Вывод 3:

<div style="background-color: #fff0ee; padding: 10px; font-family: monospace; font-size: 15px">
Есть, однако топ-категории с задержками и без находятся очень рядом друг с другом</div>

### Вопрос 4:

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Как разные цели кредита влияют на его возврат в срок?</div>

### Вывод 4:

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
С недвижимости возвращают больше кредитов, с образованием и автомобилями – меньше</div>

## 1.1.   Общий вывод:

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Каждый параметр влияет на показатели возврата кредита вовремя, портрет самого честного человека – человек в разводе, с 5 детьми и желающий взять кредит на операции с недвижимостью</div>